### Set up API Connection

In [1]:
import pandas as pd 
import numpy as np 
import datetime as dt
import time
import requests 
import json 

In [2]:
#Got this code from Brian in class 

#Set it up this way so we can easily use them to build the next function 

kind = 'submission'
subreddit = 'lifeprotips'
size = 500 

#URL - will return in JSON
f'https://api.pushshift.io/reddit/search/{kind}/?subreddit={subreddit}&size={size}'

'https://api.pushshift.io/reddit/search/submission/?subreddit=lifeprotips&size=500'

In [3]:
field = ['selftext', 'title', 'domain', 'created_utc', 'num_comments', 'subreddit', 'is_self']

### Webscraping 

In [4]:
#Got this code from Brian in class. I removed author. 

def query_pushshift (subreddit, kind = 'submission', skip=30, times=10, size=500,
                     subfield= ['selftext', 'title', 'domain', 'created_utc',
                                'num_comments', 'subreddit', 'is_self'],
                     comfield = ['body', 'created_utc']):
    
    stem = f'https://api.pushshift.io/reddit/search/{kind}/?subreddit={subreddit}&size={size}'
    
    my_list = []
    
    for x in range(1, times):
        
        URL = f'{stem}&after={skip * x}d'
        print(URL)
        response = requests.get(URL)
        assert response.status_code == 200       #if the boolean statment works them move on, but if not then send error code (instead of try and except)
        mine = response.json()['data']
        df = pd.DataFrame.from_dict(mine)        #creating a dataframe of all the data we created above 
        my_list.append(df)
        time.sleep(2)
        
    full = pd.concat(my_list, sort=False)
    
    if kind == 'submission':
        full = full[subfield]                    #so that we only see our subfields
        full = full.drop_duplicates()            #drop duplicates
        full = full.loc[full['is_self'] == True] #anytime is_self is false, there is no text. So its telling it to not take in anything where is_self is false. bc were doing NLP we want text so its pointless to ge the false ones 
        
    def get_date(created):
        return dt.date.fromtimestamp(created)
    
    _timestamp = full['created_utc'].apply(get_date)
    
    full['timestamp'] = _timestamp
    
    print(full.shape)
    
    return full 
 

In [5]:
lpt = query_pushshift('lifeprotips')

https://api.pushshift.io/reddit/search/submission/?subreddit=lifeprotips&size=500&after=30d
https://api.pushshift.io/reddit/search/submission/?subreddit=lifeprotips&size=500&after=60d
https://api.pushshift.io/reddit/search/submission/?subreddit=lifeprotips&size=500&after=90d
https://api.pushshift.io/reddit/search/submission/?subreddit=lifeprotips&size=500&after=120d
https://api.pushshift.io/reddit/search/submission/?subreddit=lifeprotips&size=500&after=150d
https://api.pushshift.io/reddit/search/submission/?subreddit=lifeprotips&size=500&after=180d
https://api.pushshift.io/reddit/search/submission/?subreddit=lifeprotips&size=500&after=210d
https://api.pushshift.io/reddit/search/submission/?subreddit=lifeprotips&size=500&after=240d
https://api.pushshift.io/reddit/search/submission/?subreddit=lifeprotips&size=500&after=270d
(4474, 8)


In [6]:
eli5 = query_pushshift('explainlikeimfive')

https://api.pushshift.io/reddit/search/submission/?subreddit=explainlikeimfive&size=500&after=30d
https://api.pushshift.io/reddit/search/submission/?subreddit=explainlikeimfive&size=500&after=60d
https://api.pushshift.io/reddit/search/submission/?subreddit=explainlikeimfive&size=500&after=90d
https://api.pushshift.io/reddit/search/submission/?subreddit=explainlikeimfive&size=500&after=120d
https://api.pushshift.io/reddit/search/submission/?subreddit=explainlikeimfive&size=500&after=150d
https://api.pushshift.io/reddit/search/submission/?subreddit=explainlikeimfive&size=500&after=180d
https://api.pushshift.io/reddit/search/submission/?subreddit=explainlikeimfive&size=500&after=210d
https://api.pushshift.io/reddit/search/submission/?subreddit=explainlikeimfive&size=500&after=240d
https://api.pushshift.io/reddit/search/submission/?subreddit=explainlikeimfive&size=500&after=270d
(4500, 8)


In [7]:
#Assign to variable and check the data 
lpt_df  = pd.DataFrame(lpt)
lpt_df.head()

,selftext,title,domain,created_utc,num_comments,subreddit,is_self,timestamp
0,,LPT: Put a sneaker on a turkey leg and let you...,self.LifeProTips,1552170236,1,LifeProTips,True,2019-03-09
1,,"LPT: Let other people explain concepts to you,...",self.LifeProTips,1552170252,112,LifeProTips,True,2019-03-09
2,[removed],Not sure where to ask this question. But I had...,self.LifeProTips,1552170315,2,LifeProTips,True,2019-03-09
3,[removed],"LPT: To peel a garlic clove, cut it lengthwise...",self.LifeProTips,1552170365,1,LifeProTips,True,2019-03-09
4,I went to Thailand on a trip and found the are...,LPT - always take a picture of the number plat...,self.LifeProTips,1552170935,1,LifeProTips,True,2019-03-09


In [8]:
#Assign to variable and check the data 
eli5_df = pd.DataFrame(eli5)
eli5_df.head()

,selftext,title,domain,created_utc,num_comments,subreddit,is_self,timestamp
0,,ELI5 can i turn my NAS in to an server? and if...,self.explainlikeimfive,1552169665,5,explainlikeimfive,True,2019-03-09
1,[removed],Why do game consoles not need a 3 prong plug w...,self.explainlikeimfive,1552169800,0,explainlikeimfive,True,2019-03-09
2,,ELI5: If your balls are outside of your body t...,self.explainlikeimfive,1552169851,1,explainlikeimfive,True,2019-03-09
3,[removed],My family utility bill is ~$400 WHY?,self.explainlikeimfive,1552170170,2,explainlikeimfive,True,2019-03-09
4,,ELI5: what’s the difference between computer g...,self.explainlikeimfive,1552170362,2,explainlikeimfive,True,2019-03-09


In [9]:
#Concatenate the data from both subreddits 
df = pd.concat([lpt, eli5])
df.head()

,selftext,title,domain,created_utc,num_comments,subreddit,is_self,timestamp
0,,LPT: Put a sneaker on a turkey leg and let you...,self.LifeProTips,1552170236,1,LifeProTips,True,2019-03-09
1,,"LPT: Let other people explain concepts to you,...",self.LifeProTips,1552170252,112,LifeProTips,True,2019-03-09
2,[removed],Not sure where to ask this question. But I had...,self.LifeProTips,1552170315,2,LifeProTips,True,2019-03-09
3,[removed],"LPT: To peel a garlic clove, cut it lengthwise...",self.LifeProTips,1552170365,1,LifeProTips,True,2019-03-09
4,I went to Thailand on a trip and found the are...,LPT - always take a picture of the number plat...,self.LifeProTips,1552170935,1,LifeProTips,True,2019-03-09


In [10]:
#Check that we have enough rows 
df.shape

(8974, 8)

In [11]:
#Export combined data to a csv (exclude the index column "Unnamed: 0")
df.to_csv('combined_reddit_data.csv', index=False)